# Prospect Theory

## Introduction

This analysis focused on fitting models to risky choices. Starting with simple expected value and expected utility models, this analysis gradually moved onto Prospect Theory (Tversky & Kahneman, 1979; 1992) with varying levels of complexity.

Here, data was analysed at an aggregated level.

The data was taken from Rieskamp (2011). Some elements are based on Walasek (2019), with modifications.

In [1]:
# loads packages

library(data.table)
library(magrittr) # makes some codes more readable

In [2]:
# loads data set

dat_gambles<-fread("Data/Data_Gambles.csv")

## Data

Participants were presented two gambles at a time, each of which had two probabilistic outcomes.

For instance, consider the first choice pair saw by the first participant:

In [3]:
dat_gambles[1, 1:8]

subject,choicepair,p1,x1,y1,p2,x2,y2
<int>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<int>
1,1,0.29,88,78,0.29,53,91


Gamble 1 had

- Outcome x: a 29% chance of winning 88 points
- Outcome y: a 100%-29% = 71% chance of winning 78 points

Gamble 2 had

- Outcome x: a 29% chance of winning 53 points
- Outcome y: a 100%-29% = 71% chance of winning 91 points

Choices were coded as 1 and 0, representing gambles 1 and 2 respectively.

In other words, the choice variable records *whether* a participant *chose gamble 1* in a choice pair.

The first participant chose gamble 2 in the above choice pair:

In [4]:
dat_gambles[1, choice]

[1] 0

## Model 1 (Expected Value)

### Mathematical Form

The subjective value of a gamble $j$, $V(g_j)$, is defined as its expected value:

$$
V(g_j)=
EV_j =
p_j \cdot x_j + (1 - p_j) \cdot y_j
$$

This model assumes individuals made choices by comparing expected values of the two gambles and the process is probabilistic.

A value function, $f(V(g_1), V(g_2))$, therefore is defined using the difference in expected values, plus a normally distributed noise term:

$$
f(V(g_1), V(g_2)) =
V(g_1) - V(g_2) + \varepsilon,  \varepsilon \sim N(0, \sigma)
$$

$$
f(V(g_1), V(g_2)) \sim N(V(g_1) - V(g_2), \sigma)
$$

It is assumed that individuals chose gamble 1 when it was percieved to have a higher subjective value than gamble 2.

Hence the probability of choosing gamble 1, $P(V(g_1), V(g_2))$, is defined as follows:

$$
P(V(g_1), V(g_2)) = P(f(V(g_1), V(g_2)) > 0) 
$$

In the following likelihood section, $P(V(g_1), V(g_2))$ is also denoted $\text{Prob}$ for the sake of convenience.

The model has one parameter in total, $\sigma$.

In [5]:
func_model_EV<-function(p1, x1, y1, p2, x2, y2, parameters){
    
    sigma<-parameters[1] # this model only has one parameter
    
    V_1<-p1*x1 + (1-p1)*y1
    V_2<-p2*x2 + (1-p2)*y2
    
    prob_gamble_1<-pnorm(0, V_1-V_2, sigma, lower.tail=FALSE)
    
    return(prob_gamble_1)
    
}

### Likelihood

Assuming each choice was an independent Bernoulli process, the likelihood of a single choice $i$ is:

$$
L(\mathit{Prob}_i | \mathit{Choice}_i) =
P(\mathit{Choice}_i | \mathit{Prob}_i) =
\mathit{Prob}_i^{\mathit{Choice}_i} (1-\mathit{Prob}_i)^{1-\mathit{Choice}_i}
$$

where $\mathit{Choice}_i$ is 1 for choosing gamble 1 and 0 for gamble 2, as stated above, while $\mathit{Prob}_i$ is the probability of choosing gamble 1 in choice pair $i$.

The log-likelihood of the model is:

$$
log L(\mathbf{Prob} | \mathbf{Choice}) = \sum log L(\mathit{Prob}_i | \mathit{Choice}_i)
$$

In [6]:
func_likelihood<-function(choice, prob_gamble_1){
    
    likelihood<-(prob_gamble_1^choice) * ((1-prob_gamble_1)^(1-choice)) # this can be simplified with an ifelse(), but this is the true mathematical form
    
    return(likelihood)
    
}

In [7]:
func_log_likelihood<-function(data, model, parameters){
    
    func_model<-model
    
    vec_prob_gamble_1<-func_model(data$p1, data$x1, data$y1, data$p2, data$x2, data$y2, parameters)
    
    vec_likelihood<-func_likelihood(data$choice, vec_prob_gamble_1)
    
    log_likelihood<-vec_likelihood %>% log %>% sum
    
    return(log_likelihood)
    
}

In [8]:
MLE_EV<-optim(1, func_log_likelihood, control=list(fnscale = -1), method="Brent", lower=-15, upper=40, data=dat_gambles, model=func_model_EV)

In [9]:
MLE_EV$par

[1] 22.8829

In [10]:
MLE_EV$value

[1] -3164.201

## Model 2 (Expected Utility)

### Mathematical Form

The subjective value of gamble $j$, $V(g_j)$, is defined as its expected utility:

$$
V(g_j) =
EU_j =
p_j \cdot U(x_j) + (1 - p_j) \cdot U(y_j)
$$

where

$$
U(z) =
\begin{cases}
z^\alpha, & z \geq 0 \\
- |z|^\alpha, & z<0
\end{cases}
$$

The value function is the same as in the expected value model:

$$
f(V(g_1), V(g_2)) =
V(g_1) - V(g_2) + \varepsilon,  \varepsilon \sim N(0, \sigma)
$$

$$
f(V(g_1), V(g_2)) \sim N(V(g_1) - V(g_2), \sigma)
$$

The probability of choosing gamble 1, $P(V(g_1), V(g_2))$, is also the same:

$$
P(V(g_1), V(g_2)) = P(f(V(g_1), V(g_2)) > 0) 
$$

The model has two parameters in total, $\alpha$ and $\sigma$.

In [11]:
func_model_EU<-function(p1, x1, y1, p2, x2, y2, parameters){
    
    alpha<-parameters[1]
    sigma<-parameters[2]
    
    func_U<-function(payoff){
        
        utility<-sign(payoff) * (abs(payoff)^alpha)
        
        return(utility)

    }
    
    V_1<-p1*func_U(x1) + (1-p1)*func_U(y1)
    V_2<-p2*func_U(x2) + (1-p2)*func_U(y2)
    
    prob_gamble_1<-pnorm(0, V_1-V_2, sigma, lower.tail=FALSE)
    
    return(prob_gamble_1)
    
}

In [12]:
MLE_EU<-optim(c(.5, 1), func_log_likelihood, control=list(fnscale = -1), data=dat_gambles, model=func_model_EU)

In [13]:
MLE_EU$par

[1]  0.9303386 16.3083384

In [14]:
MLE_EU$value

[1] -3160.305

## Model Comparison

The two models above can be compared with a likelihood ratio test:

$$
\chi^2 \approx -2 \ln L_{EV} - (-2 \ln L_{EU})
$$

with $K = 1$ degree of freedom, which is the number of extra parameter.

In [15]:
func_likelihood_ratio_test<-function(MLE_restricted, MLE_unrestricted, df){
    
    chi_2<-(-2*MLE_restricted)-(-2*MLE_unrestricted)
    
    p<-(1-pchisq(chi_2, df))
    
    return(p)
    
}

In [16]:
func_likelihood_ratio_test(MLE_EV$value, MLE_EU$value, 1)

[1] 0.005246342

## Model 3 (Simple Prospect Theory)

### Mathematical Form

The subjective value of gamble $j$, $V(g_j)$, is defined the same as the above expected utility model:

$$
V(g_j) =
EU_j =
p_j \cdot U(x_j) + (1 - p_j) \cdot U(y_j)
$$

where

$$
U(z) =
\begin{cases}
z^\alpha, & z \geq 0 \\
- |z|^\alpha, & z<0
\end{cases}
$$

The probability of choosing gamble 1, $P(V(g_1), V(g_2))$, is defined directly in a logit form as follows:

$$
P(V(g_1), V(g_2)) =
\frac{1}{1 + e^{-\epsilon(V(g_1) - V(g_2))}}
$$

The model has two parameters in total, $\alpha$ and $\epsilon$.

In [17]:
func_model_PT_simple<-function(p1, x1, y1, p2, x2, y2, parameters){
    
    alpha<-parameters[1]
    epsilon<-parameters[2]
    
    func_U<-function(payoff){
        
        utility<-sign(payoff) * (abs(payoff)^alpha)
        
        return(utility)

    }
    
    V_1<-p1*func_U(x1) + (1-p1)*func_U(y1)
    V_2<-p2*func_U(x2) + (1-p2)*func_U(y2)
    
    prob_gamble_1<-1/(1 + exp(-epsilon*(V_1-V_2)))
    
    return(prob_gamble_1)
    
}

In [18]:
MLE_PT_simple<-optim(c(1, .5), func_log_likelihood, control=list(fnscale = -1), data=dat_gambles, model=func_model_PT_simple)

In [19]:
MLE_PT_simple$par

[1] 0.9314846 0.1042756

In [20]:
MLE_PT_simple$value

[1] -3151.211

## Model 4 (Prospect Theory, Loss Aversion)

### Mathematical Form

The subjective value of gamble $j$, $V(g_j)$, is the same:

$$
V(g_j) =
p_j \cdot U(x_j) + (1 - p_j) \cdot U(y_j)
$$

the only addition is the loss aversion parameter $\lambda$:

$$
U(z) =
\begin{cases}
z^\alpha, & z \geq 0 \\
- \lambda |z|^\alpha, & z<0
\end{cases}
$$

The same choice function is defined:

$$
P(V(g_1), V(g_2)) =
\frac{1}{1 + e^{-\epsilon(V(g_1) - V(g_2))}}
$$

The model has three parameters, $\alpha$, $\lambda$, and $\epsilon$.

In [21]:
func_model_PT_LA<-function(p1, x1, y1, p2, x2, y2, parameters){
    
    alpha<-parameters[1]
    lambda<-parameters[2]
    epsilon<-parameters[3]
    
    func_U<-function(payoff){
        
        utility<-ifelse(!(payoff<0), payoff^alpha, (-lambda) * (abs(payoff)^alpha))
        
        return(utility)

    }
    
    V_1<-p1*func_U(x1) + (1-p1)*func_U(y1)
    V_2<-p2*func_U(x2) + (1-p2)*func_U(y2)
    
    prob_gamble_1<-1/(1 + exp(-epsilon*(V_1-V_2)))
    
    return(prob_gamble_1)
    
}

In [22]:
MLE_PT_LA<-optim(c(1, 1, .5), func_log_likelihood, control=list(fnscale = -1), data=dat_gambles, model=func_model_PT_LA)

In [23]:
MLE_PT_LA$par

[1] 0.9347681 1.0294685 0.1013397

In [24]:
MLE_PT_LA$value

[1] -3150.985

## Model 5 (Prospect Theory, Loss Aversion, Different Curves)

### Mathematical Form

The subjective value of gamble $j$, $V(g_j)$, is again the same:

$$
V(g_j) =
p_j \cdot U(x_j) + (1 - p_j) \cdot U(y_j)
$$

The addition is $\beta$ which allows different curve for gains and losses:

$$
U(z) =
\begin{cases}
z^\alpha, & z \geq 0 \\
-\lambda |z|^\beta, & z<0
\end{cases}
$$

The choice function is the same:

$$
P(V(g_1), V(g_2)) =
\frac{1}{1 + e^{-\epsilon(V(g_1) - V(g_2))}}
$$

The model has four parameters, $\alpha$, $\beta$, $\lambda$, and $\epsilon$.

In [25]:
func_model_PT_LA_DC<-function(p1, x1, y1, p2, x2, y2, parameters){
    
    alpha<-parameters[1]
    beta<-parameters[2]
    lambda<-parameters[3]
    epsilon<-parameters[4]
    
    func_U<-function(payoff){
        
        utility<-ifelse(!(payoff<0), payoff^alpha, (-lambda) * (abs(payoff)^beta))
        
        return(utility)

    }
    
    V_1<-p1*func_U(x1) + (1-p1)*func_U(y1)
    V_2<-p2*func_U(x2) + (1-p2)*func_U(y2)
    
    prob_gamble_1<-1/(1 + exp(-epsilon*(V_1-V_2)))
    
    return(prob_gamble_1)
    
}

In [26]:
MLE_PT_LA_DC<-optim(c(1, 1, 1, .5), func_log_likelihood, control=list(fnscale = -1), data=dat_gambles, model=func_model_PT_LA_DC)

In [27]:
MLE_PT_LA_DC$par

[1] 0.7989757 1.1002562 0.2396980 0.1997382

In [28]:
MLE_PT_LA_DC$value

[1] -3139.859

## Grid Search

Note that each maximum likelihood calculated above is based on a single set of starting values and is potentially associated with a local maxima.

From the above results and also from literature, it is expected that the MLE estimates of the parameters will be within the following ranges:

- $\alpha$: [0.5, 1.2]
- $\beta$: [0.5, 1.2]
- $\lambda$: [0, 2.5]
- $\epsilon$: [0.05, 0.15]

Therefore MLE were repeated 20 times for each model with randomised starting values.

In [29]:
n<-20

### Model 3

In [30]:
set.seed(42)

alpha<-runif(n, .5, 1.2)
epsilon<-runif(n, .05, .15)

chain_MLE_PT_simple<-integer(0)
chain_alpha_PT_simple<-integer(0)
chain_epsilon_PT_simple<-integer(0)

for (i in 1:n){
    
    MLE_PT_simple<-optim(c(alpha[i], epsilon[i]), func_log_likelihood, control=list(fnscale = -1), data=dat_gambles, model=func_model_PT_simple)
    
    chain_MLE_PT_simple<-c(chain_MLE_PT_simple, MLE_PT_simple$value)
    chain_alpha_PT_simple<-c(chain_alpha_PT_simple, MLE_PT_simple$par[1])
    chain_epsilon_PT_simple<-c(chain_epsilon_PT_simple, MLE_PT_simple$par[2])
    
}

In [31]:
chain_MLE_PT_simple

[1] -3151.211 -3151.211 -3151.211 -3151.211 -3151.211 -3151.211 -3151.211
 [8] -3151.211 -3151.211 -3151.211 -3151.211 -3151.211 -3151.211 -3151.211
[15] -3151.211 -3151.211 -3151.211 -3151.211 -3151.211 -3151.211

In [32]:
chain_alpha_PT_simple[which(chain_MLE_PT_simple==max(chain_MLE_PT_simple))]

[1] 0.9314768

In [33]:
chain_epsilon_PT_simple[which(chain_MLE_PT_simple==max(chain_MLE_PT_simple))]

[1] 0.104263

### Model 4

In [34]:
set.seed(42)

alpha<-runif(n, .5, 1.2)
lambda<-runif(n, 0, 2.5)
epsilon<-runif(n, .05, .15)

chain_MLE_PT_LA<-integer(0)
chain_alpha_PT_LA<-integer(0)
chain_lambda_PT_LA<-integer(0)
chain_epsilon_PT_LA<-integer(0)

for (i in 1:n){
    
    MLE_PT_LA<-optim(c(alpha[i], lambda[i], epsilon[i]), func_log_likelihood, control=list(fnscale = -1), data=dat_gambles, model=func_model_PT_LA)
    
    chain_MLE_PT_LA<-c(chain_MLE_PT_LA, MLE_PT_LA$value)
    chain_alpha_PT_LA<-c(chain_alpha_PT_LA, MLE_PT_LA$par[1])
    chain_lambda_PT_LA<-c(chain_lambda_PT_LA, MLE_PT_LA$par[2])
    chain_epsilon_PT_LA<-c(chain_epsilon_PT_LA, MLE_PT_LA$par[3])
    
}

In [35]:
chain_MLE_PT_LA

[1] -3150.985 -3150.985 -3150.985 -3150.985 -3150.984 -3150.985 -3150.985
 [8] -3150.984 -3150.985 -3150.985 -3150.984 -3150.984 -3150.985 -3150.985
[15] -3150.985 -3150.985 -3150.985 -3150.985 -3150.985 -3150.985

In [36]:
chain_alpha_PT_LA[which(chain_MLE_PT_LA==max(chain_MLE_PT_LA))]

[1] 0.9345633

In [37]:
chain_lambda_PT_LA[which(chain_MLE_PT_LA==max(chain_MLE_PT_LA))]

[1] 1.029536

In [38]:
chain_epsilon_PT_LA[which(chain_MLE_PT_LA==max(chain_MLE_PT_LA))]

[1] 0.1014271

### Model 5

In [39]:
set.seed(42)

alpha<-runif(n, .5, 1.2)
beta<-runif(n, .5, 1.2)
lambda<-runif(n, 0, 2.5)
epsilon<-runif(n, .05, .15)

chain_MLE_PT_LA_DC<-integer(0)
chain_alpha_PT_LA_DC<-integer(0)
chain_beta_PT_LA_DC<-integer(0)
chain_lambda_PT_LA_DC<-integer(0)
chain_epsilon_PT_LA_DC<-integer(0)

for (i in 1:n){
    
    MLE_PT_LA_DC<-optim(c(alpha[i], beta[i], lambda[i], epsilon[i]), func_log_likelihood, control=list(fnscale = -1), data=dat_gambles, model=func_model_PT_LA_DC)
    
    chain_MLE_PT_LA_DC<-c(chain_MLE_PT_LA_DC, MLE_PT_LA_DC$value)
    chain_alpha_PT_LA_DC<-c(chain_alpha_PT_LA_DC, MLE_PT_LA_DC$par[1])
    chain_beta_PT_LA_DC<-c(chain_beta_PT_LA_DC, MLE_PT_LA_DC$par[2])
    chain_lambda_PT_LA_DC<-c(chain_lambda_PT_LA_DC, MLE_PT_LA_DC$par[3])
    chain_epsilon_PT_LA_DC<-c(chain_epsilon_PT_LA_DC, MLE_PT_LA_DC$par[4])
    
}

In [40]:
chain_MLE_PT_LA_DC

[1] -3140.459 -3139.709 -3139.709 -3139.709 -3139.709 -3139.920 -3140.438
 [8] -3139.709 -3149.342 -3140.060 -3141.300 -3139.720 -3139.709 -3141.972
[15] -3139.709 -3145.652 -3139.912 -3139.713 -3139.709 -3139.751

In [41]:
chain_alpha_PT_LA_DC[which(chain_MLE_PT_LA_DC==max(chain_MLE_PT_LA_DC))]

[1] 0.787447

In [42]:
chain_beta_PT_LA_DC[which(chain_MLE_PT_LA_DC==max(chain_MLE_PT_LA_DC))]

[1] 1.126612

In [43]:
chain_lambda_PT_LA_DC[which(chain_MLE_PT_LA_DC==max(chain_MLE_PT_LA_DC))]

[1] 0.1989351

In [44]:
chain_epsilon_PT_LA_DC[which(chain_MLE_PT_LA_DC==max(chain_MLE_PT_LA_DC))]

[1] 0.2128762

## Model Comparison

### Models 3 & 4

In [45]:
func_likelihood_ratio_test(max(chain_MLE_PT_simple), max(chain_MLE_PT_LA), 1) # non-significant

[1] 0.5008979

### Models 4 & 5

In [46]:
func_likelihood_ratio_test(max(chain_MLE_PT_LA), max(chain_MLE_PT_LA_DC), 1) # significant

[1] 2.046377e-06

### Models 3 & 5

In [47]:
func_likelihood_ratio_test(max(chain_MLE_PT_simple), max(chain_MLE_PT_LA_DC), 2) # significant

[1] 1.010967e-05

## Conclusion

A partial grid search suggested that local maxima was not an issue.

Results showed that Prospect Theory models fit the data better than simpler models (the differences are rather obvious and also not the focus of this analysis, therefore were not tested), and further showed that model 5 fit better than models 3 and 4.

In short, at an aggregated level, including loss aversion and different curves for gains and losses is preferred.